# **Spotting Fake News with Python**

This project aims to investigate the task of spotting fake news and apply classic and modern machine learning techniques to develop a model that can classify news as real or fake. It is part of an assignment for the course `486205 - APRENDIZADO DE MÁQUINA` at the **Federal University of São Carlos (UFSCar)**, lectured by the professor **Tiago A. Almeida**.

---
## **Enviroment Setup and Loading Data**

The first step in this problem is to import the tools we will need further in the project. To begin the setup, we imported **pandas** and **NumPy** to manipulate the data.

News is written in a natural language that humans understand, so it is necessary to clean and prepare this data for machine processing. To satisfy our natural language processing needs throughout the project, we have decided to use the **NLTK** library. 

To improve our understanding of the task and the data we are working with, we have chosen **matplotlib** as our visualization library to clarify the extracted information from the processed news.

Compiling all the decisions, we imported the following libraries for this project:

- pandas
- NumPy
- NLTK
- matplotlib

In [ ]:
#Basic imports
import numpy as np
import pandas as pd

#NLP
import nltk

#Visualization
import matplotlib.pyplot as plt

The data we are using in this project was provided by the professor for use in the assignment. It compiles news automatically collected during 2019 and 2020 with no supervision. This indicates the possibility of data corruption or information loss during the collection phase. A given news presented in the dataset can be classified in a binary way:

- **1** -> The news is trustworthy (Real).
- **0** -> The news is not trustworthy (Possibly Fake).

However, it wasn't possible to check if the information was reliable in a large amount of samples. Thus, these news articles do not exhibit any label attached to them. The data is organized by month and year in separate CSV files, which include a unique identification for each article, both the titles and contents and the binary label attached to them. To work with this data, we imported each file and compiled them in a single data frame.

In [ ]:
files = ["news_data_01_Jan_2019.csv",
         "news_data_02_Feb_2019.csv",
         "news_data_03_Mar_2019.csv",
         "news_data_04_Apr_2019.csv",
         "news_data_05_May_2019.csv",
         "news_data_06_Jun_2019.csv",
         "news_data_07_Jul_2019.csv",
         "news_data_08_Aug_2019.csv",
         "news_data_09_Sep_2019.csv",
         "news_data_10_Oct_2019.csv",
         "news_data_11_Nov_2019.csv",
         "news_data_12_Dec_2019.csv",
         "news_data_13_Jan_2020.csv",
         "news_data_14_Feb_2020.csv",
         "news_data_15_Mar_2020.csv",
         "news_data_16_Apr_2020.csv",
         "news_data_17_May_2020.csv",
         "news_data_18_Jun_2020.csv",
         "news_data_19_Jul_2020.csv",
         "news_data_20_Aug_2020.csv",
         "news_data_21_Sep_2020.csv",
         "news_data_22_Oct_2020.csv",
         "news_data_23_Nov_2020.csv",
         "news_data_24_Dec_2020.csv",
         ]

data = pd.concat((pd.read_csv("./data/"+f) for f in files), ignore_index=True)
data.set_index('id',inplace=True)

---
## **Exploring the news and their content**

---
## **Applying machine learning to detect fake news**

---
## **Experiment Results and Comments**